<a href="https://colab.research.google.com/github/joaoppadua/adi_4277/blob/main/adi_4277.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook to clean and work with data from ADI 4277

In [2]:
!pip install stanza

     |████████████████████████████████| 342 kB 5.3 MB/s 


In [3]:
#Import modules
import re, nltk, os, stanza
from nltk import RegexpTokenizer
stanza.download('pt')

2021-09-22 20:57:53 INFO: Downloading default packages for language: pt (Portuguese)...


2021-09-22 20:58:32 INFO: Finished downloading models and saved to /root/stanza_resources.


In [4]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


First, we need do clean the data from headers and stuff. Let's write a function for that (adapted from Diego Feijo: https://github.com/diego-feijo?tab=repositories)

In [5]:
def remove_headers(text):
    '''Function to clean "acórdão" from as much textual noise as possible
      Input: string
      Output: string'''
    presentation_pattern = re.compile(r'Supremo Tribunal Federal\s+Coordenadoria de Análise de Jurisprudência\s+Dje nº \d+ Divulgação \d+\/\d+\/\d+\s+Publicação \d+\/\d+\/\d+\sEmentário nº \d+-\d\s+\d+\/\d+\/\d+\s+')
    name_pattern = re.compile(r'AÇÃO DIRETA DE INCONSTITUCIONALIDADE \d\.\d+ DISTRITO FEDERAL')
    ref_pattern = re.compile(r'\nADI 4\.277\s')
    req_pattern = re.compile(r'REQTE.+')
    int_pattern = re.compile(r'INTDO.+')
    adv_pattern = re.compile(r'ADV.+')
    date_pattern =re.compile(r'\n\d+\/\d+\/\d+\s')
    header_pattern1 = re.compile(r'(^documento pode ser acessado .*?^Inteiro Teor .*?$|^\d*\nDocumento assinado digitalmente conforme.*?$|(^[A-Z]+ \d+ [A-Z]+)* / (AC|AM|AP|RS|SC|PR|RJ|SP|ES|MG|BA|SE|AL|PE|PI|CE|RN|PA|MA|RO|RR|MA|PB|TO|MS|MT|GO|DF)$)', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    header_pattern2 = re.compile(r'(^documento pode ser acessado .*?$|^Documento assinado digitalmente conforme.*?$)', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    case_number = re.compile(r'(\n\n[A-Z]+ \d+ [A-Z]+)* / (AC|AM|AP|RS|SC|PR|RJ|SP|ES|MG|BA|SE|AL|PE|PI|CE|RN|PA|MA|RO|RR|MA|PB|TO|MS|MT|GO|DF)\n\n', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    page_number = re.compile(r'(\n)?\d{3}\n', flags=re.UNICODE | re.DOTALL | re.MULTILINE)
    text = re.sub(presentation_pattern, '', text)
    text = re.sub(name_pattern, '', text)
    text = re.sub(ref_pattern, '', text)
    text = re.sub(req_pattern, '', text)
    text = re.sub(int_pattern, '', text)
    text = re.sub(adv_pattern, '', text)
    text = re.sub(date_pattern, '', text)
    text = re.sub(header_pattern1, '', text)
    text = re.sub(header_pattern2, '', text)
    text = re.sub(case_number, '', text)
    text = re.sub(page_number, '', text)
    return text

Now we need to load the data

In [6]:
FILEPATH = 'drive/MyDrive/coding/python/local_repo/adi_4277/'
FILENAME = 'adi_4277.txt'
with open(os.path.join(FILEPATH, FILENAME), 'r', errors='ignore') as f:
    data_raw = f.read()

In [7]:
data_sans_headers = remove_headers(data_raw)

In [8]:
data_sans_headers

'PLENÁRIO\n\n\nRELATOR\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n: MIN. AYRES BRITTO\n: PROCURADORA-GERAL DA REPÚBLICA\n: PRESIDENTE DA REPÚBLICA\n: \n: CONGRESSO NACIONAL\n: CONECTAS DIREITOS HUMANOS\n: ASSOCIAÇÃO BRASILEIRA DE GAYS, LÉSBICAS E\nTRANSGÊNEROS - ABGLT\n: MARCELA CRISTINA FOGAÇA VIEIRA E OUTRO(A/S)\n: ASSOCIAÇÃO DE INCENTIVO À EDUCAÇÃO E SAÚDE DE\nSÃO PAULO\n: FERNANDO QUARESMA DE AZEVEDO E OUTRO(A/S)\n: INSTITUTO BRASILEIRO DE DIREITO DE FAMÍLIA - IBDFAM\n: RODRIGO DA CUNHA PEREIRA\n: ASSOCIAÇÃO EDUARDO BANKS\n: REINALDO JOSÉ GALLO JÚNIOR\n: CONFERÊNCIA NACIONAL DOS BISPOS DO BRASIL - CNBB\n: JOÃO PAULO AMARAL RODRIGUES E OUTRO(A/S)\n\nEMENTA: 1. ARGUIÇÃO DE DESCUMPRIMENTO DE PRECEITO\nFUNDAMENTAL\n(ADPF).\nPERDA\nPARCIAL\nDE\nOBJETO.\nRECEBIMENTO, NA PARTE REMANESCENTE, COMO AÇÃO DIRETA\nDE INCONSTITUCIONALIDADE. UNIÃO HOMOAFETIVA E SEU\nRECONHECIMENTO\nCOMO\nINSTITUTO\nJURÍDICO.\nCONVERGÊNCIA DE OBJETOS ENTRE AÇÕES DE NATUREZA\nABSTRATA.\nJULGAMENTO\nCONJUNTO.\nEncampação\ndos

Now, clean and tokenize the texts. First, we'll write a function. Then, we will apply it to the pre-cleaned data (with most of headers, page markers and other textual noise extracted).

In [9]:
def clean_text(text):
  ''' Cleans a pre-cleaned string, normalize and tokenize it'''
  tokenized = RegexpTokenizer(r'\w+').tokenize(text)
  tokens = [token.lower() for token in tokenized if token.isalpha()]
  return tokens

In [10]:
tokens = clean_text(data_sans_headers)

In [11]:
#Check to see if it worked
len(tokens), tokens[:50]

(74957,
 ['plenário',
  'relator',
  'min',
  'ayres',
  'britto',
  'procuradora',
  'geral',
  'da',
  'república',
  'presidente',
  'da',
  'república',
  'congresso',
  'nacional',
  'conectas',
  'direitos',
  'humanos',
  'associação',
  'brasileira',
  'de',
  'gays',
  'lésbicas',
  'e',
  'transgêneros',
  'abglt',
  'marcela',
  'cristina',
  'fogaça',
  'vieira',
  'e',
  'outro',
  'a',
  's',
  'associação',
  'de',
  'incentivo',
  'à',
  'educação',
  'e',
  'saúde',
  'de',
  'são',
  'paulo',
  'fernando',
  'quaresma',
  'de',
  'azevedo',
  'e',
  'outro',
  'a'])

Everything seems fine. Now, let's stem the data and build a generator and a Stanza pipeline

In [12]:
nlp = stanza.Pipeline('pt')
nlp_text = nlp(data_sans_headers)

2021-09-22 20:58:58 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2021-09-22 20:58:58 INFO: Use device: gpu
2021-09-22 20:58:58 INFO: Loading: tokenize
2021-09-22 20:59:07 INFO: Loading: mwt
2021-09-22 20:59:07 INFO: Loading: pos
2021-09-22 20:59:07 INFO: Loading: lemma
2021-09-22 20:59:07 INFO: Loading: depparse
2021-09-22 20:59:08 INFO: Done loading processors!
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at 

In [41]:
lemmas = [word.lemma for sent in nlp_text.sentences for word in sent.words]
freqdist = nltk.FreqDist(lemmas)
freqdist['literal']

11

In [42]:
bigrams = nltk.bigrams(lemmas)
#next(bigrams)

In [43]:
literal_bigram = [(w1, w2) for w1, w2 in bigrams if w1 == 'literal' or w2 == 'literal']
literal_bigram

[('mesmo', 'literal'),
 ('literal', ','),
 ('seu', 'literal'),
 ('literal', 'categorização'),
 ('previsão', 'literal'),
 ('literal', '('),
 ('interpretação', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'não'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'de'),
 ('sentido', 'literal'),
 ('literal', 'de'),
 ('expressão', 'literal'),
 ('literal', 'não'),
 ('expressão', 'literal'),
 ('literal', 'de')]

Now we will creat a nltk.Text object and print a concordance line for the word 'literal'.

In [44]:
conc_text = nltk.Text(tokens)
conc_text.concordance('literal')

Displaying 10 of 10 matches:
te como se dá já de forma até mesmo literal com ordenamentos jurídicos da comun
 do art da constituição donde a sua literal categorização com base da sociedade
eterossexuais por força da previsão literal entre homem e mulher assiste razão 
o tribunal federal df interpretação literal do texto constitucional se isso não
o em casamento adi logo a expressão literal não deixa nenhuma dúvida de que nós
es eles resultam tanto da expressão literal da lei quanto da chamada vontade do
igurar violência contra a expressão literal do texto bittencourt carlos alberto
vel dentro dos limites da expressão literal do texto rp rel min octavio gallott
o em casamento adi logo a expressão literal não deixa dúvida alguma de que nós 
como pecado nefando ou na expressão literal daqueles textos legislativos como c


As we can see, the concordance lines table does not elucidate much. It shows sentences where the idea of a "literal" intepretation or expression is a proviso on the creative interpretation that the Constitution might also allow. The prevalence of 10 in almost 75,000 tokens also does not suggest that the concept is very relevant. 